In [5]:
import pandas as pd

In [6]:
ruta_parquet_games = "datasets/data_steam_games.parquet.gz"
ruta_parquet_reviews = "datasets/data_reviews.parquet.gz"
ruta_parquet_items = "datasets/data_items.parquet.gz"

df_games = pd.read_parquet(ruta_parquet_games)
df_reviews = pd.read_parquet(ruta_parquet_reviews)
df_items = pd.read_parquet(ruta_parquet_items)

Preparación de Datos

In [7]:
# Crear una matriz de usuario-ítem
user_item_matrix = df_reviews.pivot(index='user_id', columns='item_id', values='recommend').fillna(0)

In [ ]:
# Crear una matriz de características de género por juego
game_genre_matrix = df_games[['item_id', 'genres']].set_index('item_id')

# Aplicar la codificación one-hot para los géneros
game_genre_matrix = game_genre_matrix['genres'].str.get_dummies(sep=', ')


Calcular Similitud entre Usuarios

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud coseno entre usuarios
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud del coseno entre juegos basada en género
item_item_similarity = cosine_similarity(game_genre_matrix, game_genre_matrix)
item_item_similarity_df = pd.DataFrame(item_item_similarity, index=game_genre_matrix.index, columns=game_genre_matrix.index)


Definir la Función de Recomendación

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def recomendacion_usuario(user_id: int):
    # Obtener las calificaciones del usuario dado
    user_ratings = user_item_matrix.loc[user_id]

    # Calcular la similitud entre el usuario dado y todos los demás usuarios
    user_similarities = user_similarity_df[user_id]

    # Calcular la puntuación ponderada de juegos recomendados
    recommended_scores = user_item_matrix.mul(user_similarities, axis=0).sum()

    # Filtrar juegos que el usuario ya ha calificado
    recommended_scores = recommended_scores[user_ratings == 0]

    # Ordenar juegos por puntuación
    recommended_games = recommended_scores.sort_values(ascending=False).index.tolist()[:5]

    return {"juegos_recomendados": recommended_games}


In [ ]:
def recomendacion_juego(game_id: int):
    # Obtener la fila de similitudes para el juego dado
    game_similarity_row = item_item_similarity_df.loc[game_id]

    # Ordenar juegos por similitud
    recommended_games = game_similarity_row.sort_values(ascending=False).index.tolist()[1:6]

    return {"juegos_recomendados": recommended_games}
